In [9]:
import math
import numpy as np
import pandas as pd

# holtrop and mennen method
# R_total = R_f*(k+1) + R_app + R_w + R_b + R_tr + R_a

#바꿀 파라미터의 최대 최소값 지정
def parameter_change_range():
    L_bp_min=239
    L_bp_max=241

    B_min=43
    B_max=45

    T_min=13.5
    T_max=14

    C_b_min=0.8
    C_b_max=0.85

    parameter_list=[L_bp_min,L_bp_max,B_min,B_max,T_min,T_max,C_b_min,C_b_max]

    return parameter_list

def try_list_making(a):
    L_array=np.arange(a[0],a[1],step=1)
    B_array=np.arange(a[2],a[3],step=0.2)
    T_array=np.arange(a[4],a[5],step=0.1)
    C_b_array=np.arange(a[6],a[7],step=0.005)

    L_list=L_array.tolist()
    B_list=B_array.tolist()
    T_list=T_array.tolist()
    C_b_list=C_b_array.tolist()

    return L_list,B_list,T_list,C_b_list


#L,B,T,C_b를 결정하기 위한 함수
def holtrop_and_mennen_method(a,b,c,d): #a는 L_bp,b는 B,c는 T,d는 C_b
    #실선의 제원
    L_oa=249.58
    B_full=b
    T_full=c
    LCB=2.04  #퍼센트
    LCF=-9.68  #퍼센트
    C_b=d
    C_m=0.9967
    C_w=0.4347
    C_p=C_b/C_m
    A_bulb_full=68.16
    L_bulb_full=5.22
    S_bulb_full=112.366
    WSA_full=15459.14
    A_tr_full=116.54
    h_b_full=8.94

    #scale_ratio=58
    #B_model=B_full/scale_ratio
    #T_model=T_full/scale_ratio
    #volume_model=volume_full/(scale_ratio**3)
    #A_bulb_model=A_bulb_full/(scale_ratio**2)
    #L_bulb_model=L_bulb_full/scale_ratio
    #S_bulb_model=S_bulb_full/(scale_ratio**2)
    #WSA_model=WSA_full/(scale_ratio**2)
    #A_tr_model=A_tr_full/(scale_ratio**2)
    #h_b_model=h_b_full/scale_ratio


    #model=full에서의 계산
    scale_ratio=58
    B_model=B_full
    T_model=T_full

    
    A_bulb_model=A_bulb_full
    L_bulb_model=L_bulb_full
    S_bulb_model=S_bulb_full
    WSA_model=WSA_full
    A_tr_model=A_tr_full
    h_b_model=h_b_full


    water_density=1025.9 #kg/m^3
    gravity_accel=9.81 #m/s^2
    viscous_coefficient=0.000001883
    L_bp_full=a #m
   # L_bp_model=L_bp_full/scale_ratio #m
    L_bp_model=L_bp_full
    velocity_full=14.5 #knot
    Fn_full=velocity_full*0.5144/((gravity_accel*L_bp_full)**0.5)
    Fn_model=Fn_full

    #velocity_model=velocity_full*0.5144*((1/scale_ratio)**0.5) #m/s
    velocity_model=velocity_full*0.5144
    volume_full=(B_full*(T_full+1.5)*L_bp_full)*C_b
    volume_model=volume_full
    Rn=velocity_model*L_bp_model/viscous_coefficient

    def viscous_resistance_coefficient(a): # a는 레이놀즈 넘버
        C_f=0.075/((math.log10(a)-2)**2)
        return C_f

    def wetted_surface_area(a,b,c,d,e,f,g): # a는 L_bp, b는 T, c는 B, d는 C_m, e는 C_b, f는 C_wp, g는 A_pt(벌브 단면적)
        S=a*(2*b+c)*(d**0.5)*(0.4530+0.4425*e-0.2862*d-0.003467*(c/b)+0.3696*f)+(2.38*g/e)
        return S

    def form_factor(a,b,c,d,e,f): # a는 B, b는 L_bp, c는 T, d는 displacement volume, e는 C_p, f는 LCB
        c_14=1+0.011*10 #U-shape section with Hogner stern C_stern = 10
        L_r=b*(1-e+0.06*e*f/(4*e-1))
        k_1=0.93-1+0.487118*c_14*((a/b)**1.06806)*((c/b)**0.46106)*((b/L_r)**0.121563)*((b**3/d)**0.36486)*((1-e)**(-0.604247))
        return k_1  # 1+k1

    # 윗줄까지 viscous resistance


    def c_1(a,b,c,e,f,g,h): # a는 B, b는 L, c는 T, e는 C_wp ,f는 C_p, g는 LCB, h는 displacement
        L_r=b*(1-f+0.06*f*g/(4*f- 1))
        I_e=1+89*math.exp((-((b/a)**0.80856))*((1-e)**0.30484)*((1-f-0.0225*g)**0.6367)*((L_r/a)**0.34574)*((100*h/b**3)**0.16302))
        c1=2223105*(a/b)**3.78613*(c/a)**1.07961*(90-I_e)**(-1.37565)
        return c1

    def c_2(a,b,c,d,e): # a는 A_bt, b는 B, c는 T, d는 T_f, e는 h_b
        c3=(0.56*a**1.5)/(b*c*(0.31*(a**0.5)+d-e))
        c2=math.exp(-1.89*c3**0.5)
        return c2

    def c_5(a,b,c,d): # a는 A_t, b는 B, c는 T, d는 C_m
        c5=1-(0.8*a/(b*c*d))
        return c5

    def m_1(a,b,c,d,e): # a는 L, b는 T, c는 displacement, d는 C_p, e는 B
        c16=8.07981*d-13.8673*(d)**2+6.984388*(d)**3 # C_p < 0.8이므로
        m1=(0.0140407*(a/b))-(1.75254*(((c)**(1/3))/a))-(4.79323*(e/a))-c16
        return m1

    def m_4(a): # a는 프루드 넘버
        c15=-1.69385 #L^3/Volume < 512이므로
        m4=c15*0.4*math.exp(-0.034*(a)**(-3.29))
        return m4

    def wave_making_lambda(a,b,c): # a는 C_p, b는 L, c는 B
        lamda=1.446*a-0.03*b/c   # L/B < 12이므로
        return lamda

    # 윗줄까지 Wave-Making resistance (Fn < 0.40)


    def correlation_allowance(a,b,c,d,e,f,g): # a는 L, b는 C_b, c는 A_bt, d는 B, e는 T, f는 T_f, g는 h_b
        c4=0.04 # T_f/L > 0.04이므로
        c3=(0.56*c**1.5)/(d*e*(0.31*(c**0.5)+e-g))
        c2=math.exp(-1.89*c3**0.5)
        C_a=0.006*(a+100)**(-0.16)-0.00205+0.003*((a/7.5)**0.5)*(b**4)*c2*(0.04-c4)
        return C_a

    #윗줄까지 Model-Ship Corrlation Allowance

    def additional_peressure_bulb(a,b,c,d,e,f): # a는 벌브 투영면적,b는 흘수,c는 벌브 중심 높이, d는 속도, e는 물 밀도, f는 중력가속도
        P_b=0.56*(a**0.5)/(b-1.5*c)
        F_ni=d/((f*(b-c-0.25*((a)**0.5)+0.15*(d**2)))**0.5)
        R_b=0.11*math.exp(-3*(P_b**(-2)))*(F_ni**3)*(a**1.5)*e*f/(1+(F_ni**2))
        return R_b

    def additional_pressure_transom(a,b,c,d,e,f): #a는 속도,b는 중력가속도,c는 트랜섬 잠긴 면적 ,d는 B,e는 수선면 계수,f는 밀도
        F_nt=a/((2*b*c/(d+d*e))**0.5)

        if F_nt < 5:
            c_6=0.2*(1-0.2*F_nt)
        else:
            c_6=0

        R_tr=0.5*f*(a**2)*c*c_6
        return R_tr

    #윗줄까지 additional pressure resistance

    def toal_resistance(): #a는 속도

        #점성 저항
        C_F=viscous_resistance_coefficient(Rn)
        k=form_factor(B_model,L_bp_model,T_model,volume_model,C_p,LCB) # a는 B, b는 L_bp, c는 T, d는 displacement volume, e는 C_p, f는 LCB
        wsa_predict=wetted_surface_area(L_bp_model,T_model,B_model,C_m,C_b,C_w,A_bulb_model) # a는 L_bp, b는 T, c는 B, d는 C_m, e는 C_b, f는 C_wp, g는 A_pt(벌브 단면적)
        R_v=0.5*water_density*(velocity_model**2)*C_F*(1+k)*wsa_predict
        print("점성 저항:",R_v)

        #조파저항
        C1=c_1(B_model,L_bp_model,T_model,C_w,C_p,LCB,volume_model) # a는 B, b는 L, c는 T, e는 C_wp ,f는 C_p, g는 LCB, h는 displacement
        C2=c_2(A_bulb_model,B_model,T_model,T_model,h_b_model) # a는 A_bt, b는 B, c는 T, d는 T_f, e는 h_b
        C5=c_5(A_tr_model,B_model,T_model,C_m) # a는 A_t, b는 B, c는 T, d는 C_m
        M1=m_1(L_bp_model,T_model,volume_model,C_p,B_model) # a는 L, b는 T, c는 displacement, d는 C_p, e는 B
        M4=m_4(Fn_model) # a는 프루드 넘버
        lamda=wave_making_lambda(C_p,L_bp_model,B_model) # a는 C_p, b는 L, c는 B
        R_w=C1*C2*C5*volume_model*water_density*gravity_accel*math.exp(M1*(Fn_model**(-0.9))+M4*math.cos((lamda*(Fn_model**(-2)))))
        print("조파 저항:",R_w)

        #벌브와 트랜섬에 의한 부가저항
        R_b=additional_peressure_bulb(A_bulb_model,T_model,h_b_model,velocity_model,water_density,gravity_accel) # a는 벌브 투영면적,b는 흘수,c는 벌브 중심 높이, d는 속도, e는 물 밀도, f는 중력가속도
        R_tr=additional_pressure_transom(velocity_model,gravity_accel,A_tr_model,B_model,C_w,water_density) #a는 속도,b는 중력가속도,c는 트랜섬 잠긴 면적 ,d는 B,e는 수선면 계수,f는 밀도

        print("벌브 저항:",R_b)
        print("트랜섬 저항:",R_tr)

        #모형선 실선 상관계수
        C_A=correlation_allowance(L_bp_model,C_b,A_bulb_model,B_model,T_model,0,h_b_model) # a는 L, b는 C_b, c는 A_bt, d는 B, e는 T, f는 T_f, g는 h_b
        R_a=0.5*water_density*(velocity_model**2)*C_A*wsa_predict
        print("모형선 실선 저항:",R_a)

        R_t=R_v+R_w+R_b+R_tr+R_a

        return R_t #나오는 값은 뉴턴

    total_resistance=toal_resistance()

    print("##########################################################")

    return total_resistance,volume_full

def light_weight_estimation(a,b,c): #a는 length overall, b는 breath, c는 light weight ,d, e, f는 새로운 선박의 제원
    constant=1.0
    mothership_LWT_estimation=constant*(7.2-0.0015*a)*((a**2)*b/(10**3))*1.1 #1.1은 더블 바텀의 영향으로 곱해진 상수
    constant_LWT=c/mothership_LWT_estimation

    return constant_LWT

def newship_LWT(a,b,c): #a는 새로운 길이, b는 새로운 폭, c는 constant_LWT
    new_ship_LWT= (7.2-0.0015*(a+10))*(((a+10)**2)*b/(10**3))*1.1*c
    return new_ship_LWT

def selection_of_optimum_dimension():
    parameter_list=parameter_change_range()
    L_list,B_list,T_list,C_b_list=try_list_making(parameter_list)

    constant_lwt=light_weight_estimation(249.58,43.8,18683) #길이 폭 경하중량을 수기로 입력

    optimum_resistance1=60000000
    optimum_resistance=50000000
    estimated_LWT=1

    global optimum_length
    global optimum_breadth
    global optimum_draft
    global optimum_block_coefficient

    for i in range(len(L_list)):
        for j in range(len(B_list)):
            for k in range(len(T_list)):

                resistance,volume_full_=holtrop_and_mennen_method(L_list[i],B_list[j],T_list[k],0.810)
                new_ship_LWT=newship_LWT(L_list[i],43.8,constant_lwt)

                if (-new_ship_LWT+(volume_full_*1.0259))>320000:
                    resistance=50000000
                else:
                    optimum_resistance = resistance

                if optimum_resistance < optimum_resistance1:
                    optimum_resistance1 = optimum_resistance
                    estimated_LWT=new_ship_LWT
                    estimated_DWT=volume_full_*1.0259-estimated_LWT
                    optimum_length=round(L_list[i],3)
                    optimum_breadth=43.8
                    optimum_draft=13.538
                    optimum_block_coefficient=0.810
                    
    print("finish")

    print("##########################################################")
    print("##################  selected dimension ###################")
    print("##########################################################")

    print("Length between perpendicular:",optimum_length)
    print("Breadth:",optimum_breadth)
    print("Draft:",optimum_draft)
    print("Block coefficient:",optimum_block_coefficient)
    print("total resistance:",optimum_resistance1)
    print("Light weight:",estimated_LWT)
    print("Dead weight:",estimated_DWT)


selection_of_optimum_dimension()


점성 저항: 681673.6141159895
조파 저항: 22998.264211192654
벌브 저항: 154571.56419919728
트랜섬 저항: 502163.6116229597
모형선 실선 저항: 102351.66009607629
##########################################################
점성 저항: 683875.9639294359
조파 저항: 23148.16215303729
벌브 저항: 152337.51441689252
트랜섬 저항: 502163.6116229597
모형선 실선 저항: 102651.11322993734
##########################################################
점성 저항: 686077.103433619
조파 저항: 23297.23719495733
벌브 저항: 149726.58984565496
트랜섬 저항: 502163.6116229597
모형선 실선 저항: 102950.45800269539
##########################################################
점성 저항: 688277.0602088268
조파 저항: 23445.523268131725
벌브 저항: 146759.18488720938
트랜섬 저항: 502163.6116229597
모형선 실선 저항: 103249.69677002648
##########################################################
점성 저항: 690475.8610884118
조파 저항: 23593.05256397346
벌브 저항: 143458.21924544606
트랜섬 저항: 502163.6116229597
모형선 실선 저항: 103548.83181981769
##########################################################
점성 저항: 684230.5171218177
조파 저항: 22957.984587